In [1]:
from datetime import datetime
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from dotenv import dotenv_values
from sklearn.ensemble import ExtraTreesClassifier
pd.set_option('display.max_columns', None)  # or 1000
pd.set_option('display.max_rows', 10)  # or 1000
pd.set_option('display.max_colwidth', None)  # or 199
from data.preprocessing import DataPreprocessor
from config import get_connection
from data.db import execute_sql, batched_read_notification_sql
from data.notifcation_preparation import *

In [2]:
conf = dotenv_values("../.env")

In [3]:
conn = get_connection(conf)

In [ ]:
notifications = batched_read_notification_sql(conn, subset=15)

Running batch id >= 0 and id < 1000


In [ ]:
len(notifications)

In [ ]:
notifications_list = flat_notifications_from_sql(notifications)

res = prepare_dataset(notifications_list)

In [ ]:
res = DataPreprocessor.remove_partial_data(res)

In [ ]:
raw = pd.DataFrame(data=res)
df = raw.reindex(sorted(raw.columns), axis=1)
df.shape

In [ ]:
raw.columns

In [ ]:
df = DataPreprocessor.remove_corrupt_data(df)
df.shape


In [ ]:
# remove semi missing data 2022-05-11 to 2022-05-24
df = DataPreprocessor.remove_date_range(df,
                                        datetime.fromisoformat("2022-05-11"),
                                        datetime.fromisoformat("2022-05-24")
                                        )
df.shape

In [ ]:
# remove semi missing data 2022-06-30 16:30 to 2022-07-02 11:17
df = DataPreprocessor.remove_date_range(df,
                                        datetime.fromisoformat("2022-06-30 16:30"),
                                        datetime.fromisoformat("2022-07-02 11:17")
                                        )
df.shape

In [ ]:
df = DataPreprocessor.remove_most_recent_data(df)
df.shape

In [ ]:
df = DataPreprocessor.add_regression_label_columns(df)
df.shape

In [ ]:
df = DataPreprocessor.drop_categorical_features(df)
df.shape

In [ ]:
#df.select_dtypes(include=['object'])

In [ ]:
df = df.drop_duplicates(keep='first')
df.shape

In [ ]:
# COMMENTED OUT
# drop columns with duplicated values, leave first one
#df = df.loc[:,~df.apply(lambda y: y.duplicated(),axis=1).all()].copy()
#df.shape
#df = DataPreprocessor.select_only_required_features(df)
#df.shape

In [ ]:
current_hourly_bars_closes = [col for col in df if col.startswith('current_hour_bars') and col.endswith('close')]

current_hourly_bars_closes = sorted(current_hourly_bars_closes, reverse=True)
current_hourly_bars_closes.append("latest_hour_close")

assert len(current_hourly_bars_closes) == 49
assert current_hourly_bars_closes[-2] == 'current_hour_bars_01_close'

len(current_hourly_bars_closes)

In [ ]:
current_hourly_bars_highs = [col for col in df if col.startswith('current_hour_bars') and col.endswith('high')]

current_hourly_bars_highs = sorted(current_hourly_bars_highs, reverse=True)
current_hourly_bars_highs.append("latest_hour_close")

assert len(current_hourly_bars_highs) == 49
assert current_hourly_bars_highs[-2] == 'current_hour_bars_01_high'

len(current_hourly_bars_highs)

In [ ]:
current_hourly_bars_lows = [col for col in df if col.startswith('current_hour_bars') and col.endswith('low')]

current_hourly_bars_lows = sorted(current_hourly_bars_lows, reverse=True)
current_hourly_bars_lows.append("latest_hour_close")

assert len(current_hourly_bars_lows) == 49
assert current_hourly_bars_lows[-2] == 'current_hour_bars_01_low'

len(current_hourly_bars_lows)


In [ ]:
higher_high_cols_shift_1 = DataPreprocessor.add_higher_high_col(df)
df.loc[:, higher_high_cols_shift_1].tail()

In [ ]:
higher_high_cols_shift_5 = DataPreprocessor.add_higher_high_col(df, 5)
df.loc[:, higher_high_cols_shift_5].tail()

In [ ]:
higher_high_cols_shift_10 = DataPreprocessor.add_higher_high_col(df, 10)
df.loc[:, higher_high_cols_shift_10].tail()

In [ ]:
higher_high_cols_shift_20 = DataPreprocessor.add_higher_high_col(df, 20)
df.loc[:, higher_high_cols_shift_20].tail()

In [ ]:
higher_high_cols_shift_40 = DataPreprocessor.add_higher_high_col(df, 40)
df.loc[:, higher_high_cols_shift_40].tail()

In [ ]:
check = higher_high_cols_shift_1 + ['latest_hour_close', 'current_hour_bars_01_close']
df.loc[:, check].tail()

In [ ]:
#checks that all cols are not empty even if underlying data is nan
df.loc[:, check].loc[df['current_hour_bars_01_close'].isna()].tail()

In [ ]:
current_hourly_bars_closes

In [ ]:
# manual EMA calculation
# df['EMA_HOUR_25'] = df.loc[:, current_hourly_bars_closes].ewm(span=25, ignore_na=True, axis=1).mean()['latest_hour_close']
# df['EMA_HOUR_49'] = df.loc[:, current_hourly_bars_closes].ewm(span=49, ignore_na=True, axis=1).mean()['latest_hour_close']
# ewm25
# df.loc[:,['EMA_HOUR_25', 'EMA_HOUR_49', 'price']]

In [ ]:
import pandas_ta as pta
def pta_ema(row): 
  return pta.ema(row, length = 25)
ema = df[current_hourly_bars_closes].apply(pta_ema, axis=1, result_type='expand')

In [ ]:
import ta
from ta.trend import ema_indicator
def ta_ema(row): 
  return ema_indicator(row, window = 25)
ema_ta = df[current_hourly_bars_closes].apply(ta_ema, axis=1, result_type='expand')

In [ ]:
from indicators.momentum import willr
from functools import partial
df.apply(
    partial(willr, high_cols=current_hourly_bars_highs, low_cols=current_hourly_bars_lows, close_cols=current_hourly_bars_closes), 
    axis=1, 
    result_type='expand'
).tail()

In [ ]:
from indicators.core import add_indicator_to_df
import indicators.trend as it
import indicators.momentum as im

In [ ]:
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_closes],
                    func=partial(it.ema, length=20),
                    prefix="EMA_20_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=10)
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_closes],
                    func=partial(it.ema, length=40),
                    prefix="EMA_40_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=5)

In [ ]:
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_closes],
                    func=partial(it.dema, length=30),
                    prefix="DEMA_30_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=5)
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_closes],
                    func=partial(it.dema, length=15),
                    prefix="DEMA_15_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=10)

In [ ]:
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_closes],
                    func=partial(im.rsi, length=30),
                    prefix="RSI_30_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=5)
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_closes],
                    func=partial(im.rsi, length=15),
                    prefix="RSI_15_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=10)

In [ ]:
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_closes],
                    func=partial(im.rapo),
                    prefix="RAPO_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=10)
# df = add_indicator_to_df(base_df=df,
#                     subset_df=df[current_hourly_bars_closes],
#                     func=partial(im.rapo, length=15),
#                     prefix="RAPO_15_MINUS",
#                     postfix="HOUR",
#                     number_of_cols_to_add=10)

In [ ]:
df= add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_highs + current_hourly_bars_lows + current_hourly_bars_closes],
                    func=partial(im.willr, length=30, high_cols=current_hourly_bars_highs, low_cols=current_hourly_bars_lows, close_cols=current_hourly_bars_closes),
                    prefix="WILLR_30_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=5)
df = add_indicator_to_df(base_df=df,
                    subset_df=df[current_hourly_bars_highs + current_hourly_bars_lows + current_hourly_bars_closes],
                    func=partial(im.willr, length=15, high_cols=current_hourly_bars_highs, low_cols=current_hourly_bars_lows, close_cols=current_hourly_bars_closes),
                    prefix="WILLR_15_MINUS",
                    postfix="HOUR",
                    number_of_cols_to_add=10)

In [ ]:
df.tail()

In [ ]:
df.shape

In [ ]:
ema_ta.iloc[:, -10:].columns

In [ ]:
df = DataPreprocessor.drop_minutely_bar_cols(df)
df.shape

In [ ]:
df = DataPreprocessor.drop_hourly_bar_ohl_cols(df)
df.shape

In [ ]:
df = DataPreprocessor.drop_btc_stats_map_ol_cols(df)
df.shape

In [ ]:
df = DataPreprocessor.drop_history_stats_map_ohl_cols(df)
df.shape

In [ ]:
df = DataPreprocessor.drop_highly_correlated_features(df)
df.shape

In [ ]:
df = DataPreprocessor.drop_highly_missing_features(df)
df.shape

In [ ]:
corr_matrix = df.corr().abs()

#the matrix is symmetric so we need to extract upper triangle matrix without diagonal (k = 1)

sol = (corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
                  .stack()
                  .sort_values(ascending=False))

#first element of sol series is the pair with the biggest correlation
check_cols = set()
for index, value in sol.items():
    # do some staff
    if value > 0.9:
        #print(index, value)
        for v in index:
            check_cols.add(v)

In [ ]:
check_cols = sorted(check_cols)

In [ ]:
check_cols

In [ ]:
df.loc[:, check_cols].tail()

In [ ]:
df.tail()

In [ ]:
((df.isnull() | df.isna()).sum() * 100 / df.index.size).round(2)

In [ ]:
plt.figure(figsize=(12,12))

sns.heatmap(df.isnull(),cbar=False)

In [ ]:
# Initial try, with filling df with 0
df = df.fillna(0)

In [ ]:
# if volume is negative - replace with zero
DataPreprocessor.replace_negative_volumes(df)

In [ ]:
#already added as 'latest_hour_volume'
#DataPreprocessor.add_current_hour_volume(df)

In [ ]:
history_vol_cols = DataPreprocessor.add_current_hour_volume_to_historical_volumes_coef(df)

In [ ]:
next(history_vol_cols)

In [ ]:
# finding corrupt data with excessive volumes, requires categorical features
#df[(df["id"] > 13000) & (df["id"] < 17000)].loc[::, ["id", "notification_date", *next(history_vol_cols)]].nlargest(n=3000, columns=['current_h_vol_to_28_days_avg'], keep='all')

In [ ]:
# Which signal made best return?
df[df['LABEL_UP_RETURN'] == df['LABEL_UP_RETURN'].max()].loc[:, ['price', 'LABEL_UP_RETURN']]

In [ ]:
# Which signal made worst return?
df[df['LABEL_DOWN_RETURN'] == df['LABEL_DOWN_RETURN'].min()].loc[:, ['price', 'LABEL_DOWN_RETURN']]

In [ ]:
df_vol = df[[*next(history_vol_cols), 'LABEL_UP_RETURN', 'LABEL_DOWN_RETURN']]
x = df_vol.drop(['LABEL_UP_RETURN','LABEL_DOWN_RETURN'], 1)

# cleaning outliers in data
df_vol_coef_clean = df_vol[((x > x.quantile(.03)) & (x < x.quantile(.97))).all(1)]
#df_vol_coef_clean = df_vol[(np.abs(stats.zscore(df_vol)) < 4).all(axis=1)]
df_vol_coef_clean.shape

In [ ]:
sns.heatmap(df_vol_coef_clean.corr())

In [ ]:
df = next(DataPreprocessor.remove_outliers(df, history_vol_cols))
df.shape

In [ ]:
# removing those which don't have -3 hours of data
df = DataPreprocessor.remove_rows_with_less_than_3_hours(df)
df.shape

In [ ]:
df_change_since_previous = next(DataPreprocessor.add_change_since_1_2_3_hours_back(df))
sns.heatmap(df_change_since_previous.corr())

In [ ]:
# there is some reverse correlation in change and down return
sns.scatterplot(df['CHANGE_SINCE_01_HOUR_BARS'], df['LABEL_DOWN_RETURN'])

In [ ]:
df_volume_on_previous = df[['current_hour_bars_01_volume',
    'current_hour_bars_02_volume',
    'current_hour_bars_03_volume',
    'LABEL_UP_RETURN', 'LABEL_DOWN_RETURN']]
sns.heatmap(df_volume_on_previous.corr())

In [ ]:
DataPreprocessor.add_1_2_3_h_bars_vol_to_history_vol_coef(df)
df_volume_on_previous_as_coef = df[['_01_H_BARS_VOL_TO_28D_AVG_H_VOL',
    '_02_H_BARS_VOL_TO_28D_AVG_H_VOL',
    '_03_H_BARS_VOL_TO_28D_AVG_H_VOL',
    'LABEL_UP_RETURN', 'LABEL_DOWN_RETURN']]
sns.heatmap(df_volume_on_previous_as_coef.corr())

In [ ]:
history_change_rate_cols = [c for c in df.columns if str(c).startswith('history_statsMap_-') and str(c).endswith('changeRate')]
df_history_change_rate = df[[*history_change_rate_cols, 'LABEL_UP_RETURN', 'LABEL_DOWN_RETURN']]
sns.heatmap(df_history_change_rate.corr())

In [ ]:
df_vol_coef_clean[df_vol_coef_clean['CURRENT_H_VOL_TO_5_DAYS_AVG'] == df_vol_coef_clean['CURRENT_H_VOL_TO_5_DAYS_AVG'].max()]

In [ ]:
with_cat = df
with_cat['up_return'] = pd.cut(df['LABEL_UP_RETURN'],
                               bins=[0, 30, 40, 50, 70, 100, 300, float('Inf')], 
                               labels=['0-30', '30-40', '40-50', '50-70', '70-100', '100-300', '300+'])

In [ ]:
with_cat['down_return'] = pd.cut(df['LABEL_DOWN_RETURN'],
                                 bins=[float('-Inf'), -30, -20, -10, 0], 
                                 labels=['-30', '-30-20', '-20-10', '-10-0'])

In [ ]:
sns.displot(df_vol_coef_clean, x='current_h_vol_to_5_days_avg', bins=10)

In [ ]:
sns.boxplot(x = 'up_return', y = 'current_hour_bars_01_volume', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'current_hour_bars_02_volume', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'current_hour_bars_03_volume', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'change_since_01_hour_bars', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'change_since_02_hour_bars', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'change_since_03_hour_bars', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'down_return', y = 'change_since_01_hour_bars', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'down_return', y = 'change_since_02_hour_bars', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'down_return', y = 'change_since_03_hour_bars', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'current_h_vol_to_5_days_avg', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'current_h_vol_to_14_days_avg', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'up_return', y = 'current_h_vol_to_28_days_avg', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'down_return', y = 'current_h_vol_to_5_days_avg', data = with_cat, showfliers = False) 

In [ ]:
sns.boxplot(x = 'down_return', y = 'current_h_vol_to_28_days_avg', data = with_cat, showfliers = False) 

In [ ]:
df_vol_coef_clean.nlargest(n=20, columns=['current_h_vol_to_5_days_avg'], keep='all')

In [ ]:
# 10 largest by up return
df_vol_coef_clean.nlargest(n=20, columns=['label_up_return'], keep='all')

In [ ]:
# 10 smallest by down return
df_vol_coef_clean.nsmallest(n=20, columns=['label_down_return'], keep='all')

In [ ]:
# for c in  history_vol_cols:
#     sns.displot(df_vol_coef_clean[c], stat = 'density', binwidth=3)

In [ ]:
#list(df.columns)

In [ ]:
from data.scalers import min_max_scaler, standard_scaler


In [ ]:
df = df.drop(['up_return', 'down_return'], axis=1)
df_norm = min_max_scaler(df)
df_std= standard_scaler(df)
##%%
#corr = df.corr()['label_up_return']
##%%
#pd.set_option('display.max_rows', None)  # or 1000
#abs(corr).sort_values(ascending=False)
##%%
#pd.set_option('display.max_rows', 10)  # or 1000

In [ ]:
label_cols = ['label_up_return', 'label_down_return']
x = df.drop(label_cols, axis=1)
y_up = df['label_up_return']
y_down = df['label_down_return']

x_norm = df_norm.drop(label_cols, axis=1)
y_up_norm = df_norm['label_up_return']
y_down_norm = df_norm['label_down_return']

x_std = df_std.drop(label_cols, axis=1)
y_up_std = df_std['label_up_return']
y_down_std = df_std['label_down_return']

In [ ]:
# EDA using SWEETVIZ automl library
#import sweetviz as sv
#sv_report = sv.analyze(df, pairwise_analysis='off')
#sv_report.show_html("sv.html")

In [ ]:
from model_factory import ModelFactory
_, y_up_class = ModelFactory.prepare_classification_label(df, 20)
y_up_class.value_counts()

In [ ]:
_, y_down_class = ModelFactory.prepare_classification_label(df, -10)
y_down_class.value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

def train_random_forest_clasifier(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    rfc = ExtraTreesClassifier()
    rfc.fit(x_train, y_train)
    return x_train, x_test, y_train, y_test, rfc

In [ ]:
def train_logistic_regression_clasifier(x, y):
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)
    from sklearn.linear_model import LogisticRegression
    rfc = LogisticRegression(max_iter=5000, penalty="l2")
    rfc.fit(x_train, y_train)
    return x_train, x_test, y_train, y_test, rfc

In [ ]:
x_up_class, x_up_test_class, y_train_up_class, y_up_test_class, up_random_forest_model = train_random_forest_clasifier(x, y_up_class)

In [ ]:
def eval_model(x_up_class, x_up_test_class, y_up_class, y_up_test_class, up_random_forest_model):
    # Make predictions for the test UP set
    y_up_predictions = up_random_forest_model.predict(x_up_test_class)
    # View accuracy score
    print(accuracy_score(y_up_test_class, y_up_predictions))
    
    # View confusion matrix for test data and predictions
    matrix = confusion_matrix(y_up_test_class, y_up_predictions)
    print(matrix)
    matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]
    
    # Build the plot
    plt.figure(figsize=(16,7))
    sns.set(font_scale=1.4)
    sns.heatmap(matrix, annot=True, annot_kws={'size':10},
                cmap=plt.cm.Greens, linewidths=0.2)
    
    # Add labels to the plot
    class_names = ['Higher than 20%', 'Lower Than 20%']
    tick_marks = np.arange(len(class_names))
    tick_marks2 = tick_marks + 0.5
    plt.xticks(tick_marks, class_names, rotation=25)
    plt.yticks(tick_marks2, class_names, rotation=0)
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.title('Confusion Matrix for Random Forest Model')
    plt.show()
    # View the classification report for test data and predictions
    print(classification_report(y_up_test_class, y_up_predictions))
    

In [ ]:
eval_model(x_up_class, x_up_test_class, y_train_up_class, y_up_test_class, up_random_forest_model)

In [ ]:
from sklearn.tree import export_graphviz
import os 
from sklearn import tree
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)
tree.plot_tree(up_random_forest_model.estimators_[0],
feature_names=x_up_class.columns,
class_names=y_train_up_class.columns,
filled=True,
rounded=True)
fig.savefig('rf_individualtree.png')

In [ ]:
importances = up_random_forest_model.feature_importances_
std = np.std([tree.feature_importances_ for tree in up_random_forest_model.estimators_], axis=0)


In [ ]:
forest_importances = pd.Series(importances, index=x_up_class.columns)
forest_importances.nlargest(30).plot(kind='barh')

In [ ]:
pd.set_option('display.max_rows', 50)  # or 1000

forest_importances.nlargest(50)

In [ ]:
eval_model(*train_logistic_regression_clasifier(x_std, y_up_class))

In [ ]:
x_down_class, x_down_test_class, y_down_train_class, y_down_test_class, down_random_forest_model = train_random_forest_clasifier(x, y_down_class)

In [ ]:
# Make predictions for the test DOWN set
y_down_predictions = down_random_forest_model.predict(x_down_test_class)
# View accuracy score
accuracy_score(y_down_test_class, y_down_predictions)

In [ ]:
# View confusion matrix for test data and predictions
matrix = confusion_matrix(y_down_test_class, y_down_predictions)
matrix

In [ ]:
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

# Build the plot
plt.figure(figsize=(16,7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['Lower than -10%', 'Higher Than -10%']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()

In [ ]:
forest_importances = pd.Series(down_random_forest_model.feature_importances_, index=x_down_class.columns)
forest_importances.nlargest(50)

In [ ]:
# View the classification report for test data and predictions
print(classification_report(y_down_test_class, y_down_predictions))

In [ ]:
# check stats using OLS from statsmodels
#import statsmodels.api as sm
#results = sm.OLS(y_up,x).fit()
#results.summary()

